<a href="https://colab.research.google.com/github/apziko/NLP_BigQ/blob/main/NLP_BigQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# !python -m spacy download en_core_web_sm

In [ ]:
en_nlp = spacy.load('en_core_web_sm')

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "imdb-362914-XXXX.json"
from google.cloud import bigquery
client = bigquery.Client(project='imdb-362914')

In [ ]:
query_job = client.query(
    """
    SELECT
      review,
      IF((sentiment = 'positive'), 1, 0) AS val
    FROM
      imdb.reviews
    """
)

In [ ]:
results = query_job.result()

In [ ]:
df = results.to_dataframe()

In [ ]:
import re

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50001 entries, 0 to 50000
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  50001 non-null  object
 1   val     50001 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [ ]:
test = df['review'][1]

In [ ]:
test

"This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 or 8 years were brilliant, but things dropped off after that. By 1990, the show was not really funny anymore, and it's continued its decline further to the complete waste of time it is today.<br /><br />It's truly disgraceful how far this show has fallen. The writing is painfully bad, the performances are almost as bad - if not for the mildly entertaining respite of the guest-hosts, this show probably wouldn't still be on the air. I find it so hard to believe that the same creator that hand-selected the original cast also chose the band of hacks that followed. How can one recognize such brilliance and then see fit to replace it with such mediocrity? I felt I must give 2 stars out of respect for the original cast that made this show such a huge success. As it is now, the show is just awful. I can't believe it's still on the air."

In [ ]:
test = re.sub('<br /><br />', '', test)

In [ ]:
test

"This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 or 8 years were brilliant, but things dropped off after that. By 1990, the show was not really funny anymore, and it's continued its decline further to the complete waste of time it is today.It's truly disgraceful how far this show has fallen. The writing is painfully bad, the performances are almost as bad - if not for the mildly entertaining respite of the guest-hosts, this show probably wouldn't still be on the air. I find it so hard to believe that the same creator that hand-selected the original cast also chose the band of hacks that followed. How can one recognize such brilliance and then see fit to replace it with such mediocrity? I felt I must give 2 stars out of respect for the original cast that made this show such a huge success. As it is now, the show is just awful. I can't believe it's still on the air."

In [ ]:
df['review'] = [re.sub('<br /><br />', '', x) for x in df['review']]


In [ ]:
df = df.iloc[15000:35000, :]

In [ ]:
df['val'].value_counts()

0    10000
1    10000
Name: val, dtype: int64

In [ ]:
# docs = list(en_nlp.pipe(corpus_en))

In [ ]:
# df.to_csv('/content/drive/MyDrive/imdb_sent_2.csv', index=False)  

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from sklearn.model_selection import train_test_split
X = df['review']
y = df['val'] # przewidujemy label

X_train_, X_test, y_train_, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_, test_size=0.2, random_state=42, stratify=y_train_)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [ ]:
# tfidf.get_params()

In [ ]:
X_train_tf = tfidf.transform(X_train)
X_val_tr = tfidf.transform(X_val)
X_test_tf = tfidf.transform(X_test)

In [ ]:
X_train_tf.toarray()
X_val_tr.toarray()
X_test_tf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_tf, y_train)

LogisticRegression()

In [ ]:
y_pred = lr.predict(X_test_tf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      2000
           1       0.87      0.90      0.88      2000

    accuracy                           0.88      4000
   macro avg       0.88      0.88      0.88      4000
weighted avg       0.88      0.88      0.88      4000



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train_tf, y_train)

y_pred = neigh.predict(X_test_tf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.83      0.77      2000
           1       0.79      0.67      0.72      2000

    accuracy                           0.75      4000
   macro avg       0.75      0.75      0.75      4000
weighted avg       0.75      0.75      0.75      4000

